# CS 109A/STAT 121A/AC 209A/CSCI E-109A

## Standard Section 6: Logistic Regression and Bootstrapping

**Harvard University**<br/>
**Fall 2017**<br/>
**Section Leaders: Albert Wu, Nathaniel Burbank<br/>**
**Instructors: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine** 

<center>**Download this notebook from the CS109 repo or here:**</center>
<center>**http://bit.ly/109_S6**</center>

In this section we will be covering logistic regression using a cancer dataset. The goal is to get you familiarized with the implementation of logistic regresison and bootstrapping through the use of a dataset we haven't seen. The exercises below will help you be able to answer parts of Homework 5.

Specifically, we will: 
    
    1. Load in and get a feel for the prostate cancer dataset, which is similar in structure to our homework.
    2. Fit a logistic regression model to only one predictor and understand how to interpret the results.
    3. Compare to a linear regression model and then fit the logistic regression model to the entire dataset.
    3. Take the results from the logistic regression model to create a bootstrap with confidence intervals.


For this section we will be using the following packages:

In [128]:
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.width', 1000)
pd.set_option('display.notebook_repr_html', True)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.api import OLS
from statsmodels.api import add_constant

from statsmodels.regression.linear_model import RegressionResults
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
# Note --  Requires sklearn version .18 or higher  

from sklearn.metrics import r2_score
from collections import Counter
sns.set(style="ticks")
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

matplotlib.rcParams['figure.figsize'] = (13.0, 6.0)

assert(sys.version_info.major==3),print(sys.version)
# Python 3 or higher is required

## Warmup – back to the titanic dataset

In [129]:
import statsmodels.formula.api as sm
titanic = sns.load_dataset("titanic")
titanic.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class    who  adult_male deck  embark_town alive  alone
0         0       3    male  22.0      1      0   7.2500        S  Third    man        True  NaN  Southampton    no  False
1         1       1  female  38.0      1      0  71.2833        C  First  woman       False    C    Cherbourg   yes  False
2         1       3  female  26.0      0      0   7.9250        S  Third  woman       False  NaN  Southampton   yes   True
3         1       1  female  35.0      1      0  53.1000        S  First  woman       False    C  Southampton   yes  False
4         0       3    male  35.0      0      0   8.0500        S  Third    man        True  NaN  Southampton    no   True

In [130]:
# Subset to only cols we want
titanic = titanic[['sex', 'age', 'class', 'survived']]

# Rename 'class' col to 'pclass' to avoid namespace issues
titanic.columns = ['sex', 'age', 'pclass', 'survived']

#Drop any row with NA values in any col in dataframe
titanic = titanic.dropna()

is_female = pd.get_dummies(titanic.sex)['female']
pclass_coded = pd.get_dummies(titanic.pclass)[['First','Second']]

titanic_c = pd.concat([is_female,pclass_coded,titanic[['age','survived']]],axis=1)
titanic_c.head()

female  First  Second   age  survived
0       0      0       0  22.0         0
1       1      1       0  38.0         1
2       1      0       0  26.0         1
3       1      1       0  35.0         1
4       0      0       0  35.0         0

In [131]:
train, test =  train_test_split(titanic_c, test_size=.2, random_state=123)
model = sm.logit(formula="survived ~ female + First + Second + age", data=train)
model = model.fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.443915
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                  571
Model:                          Logit   Df Residuals:                      566
Method:                           MLE   Df Model:                            4
Date:                Tue, 17 Oct 2017   Pseudo R-squ.:                  0.3441
Time:                        21:05:51   Log-Likelihood:                -253.48
converged:                       True   LL-Null:                       -386.45
                                        LLR p-value:                 2.391e-56
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3532      0.282     -4.801      0.000      -1.906      -0.801
female         2.6625      0.238     11.207      0.000       2.197       3.128
First          2.5966      0.314      8.276      0.000       1.982       3.212
Second         1.1384      0.279      4.080      0.000       0.591       1.685
age           -0.0360      0.009     -4.124      0.000      -0.053      -0.019
==============================================================================
"""

Note that all our predictors are significant here, and none of the confidence intervals include zero. 

### Interpreting logistic regression 

Being a first class passanger increases the *logit of the probability* of surviving by **X**, relative to being a third-class passanger. 

In [132]:
model.params.First

2.5966496929609466

The odds of First class passengers surviving are **X** times more likely than the odds of a third-class passenger surviving. 

In [133]:
np.exp(model.params.First)

13.418705856625333

Calculate the probability of each passenger in our training set surviving, without using model.predict 

Hint:
$$ p = \cfrac{e^\eta}{1+e^\eta} =  \cfrac{1}{1+e^{-\eta}}  $$

Where, $\eta = \alpha + \beta_1 X_1 + ... + \beta_n X_n$

In [134]:
x_train = titanic_c[['female','First','Second','age']]
eta = model.params.values[0] + np.sum(model.params.values[1:]*x_train,axis=1)
1/(1+np.exp(-eta))

0      0.104866
1      0.926865
2      0.592506
3      0.933848
4      0.068387
6      0.332193
7      0.193857
8      0.583796
9      0.874816
10     0.762313
11     0.860607
12     0.111811
13     0.059773
14     0.691219
15     0.615377
16     0.193857
18     0.548484
20     0.186437
21     0.191952
22     0.683492
23     0.558879
24     0.735279
25     0.485715
27     0.636506
30     0.451434
33     0.069914
34     0.558879
35     0.433695
37     0.108289
38     0.659709
39     0.691219
40     0.467777
41     0.814086
43     0.912117
44     0.651591
49     0.659709
50     0.167294
51     0.108289
52     0.895103
53     0.802955
54     0.250900
56     0.844557
57     0.084865
58     0.906179
59     0.148204
60     0.104866
61     0.926865
62     0.407414
63     0.182865
66     0.802955
67     0.115432
68     0.667734
69     0.092112
70     0.203354
71     0.675663
72     0.274895
73     0.092112
74     0.075588
75     0.095163
78     0.439138
79     0.557372
80     0.104866
81     0

In [135]:
model.predict(x_train)

0      0.104866
1      0.926865
2      0.592506
3      0.933848
4      0.068387
6      0.332193
7      0.193857
8      0.583796
9      0.874816
10     0.762313
11     0.860607
12     0.111811
13     0.059773
14     0.691219
15     0.615377
16     0.193857
18     0.548484
20     0.186437
21     0.191952
22     0.683492
23     0.558879
24     0.735279
25     0.485715
27     0.636506
30     0.451434
33     0.069914
34     0.558879
35     0.433695
37     0.108289
38     0.659709
39     0.691219
40     0.467777
41     0.814086
43     0.912117
44     0.651591
49     0.659709
50     0.167294
51     0.108289
52     0.895103
53     0.802955
54     0.250900
56     0.844557
57     0.084865
58     0.906179
59     0.148204
60     0.104866
61     0.926865
62     0.407414
63     0.182865
66     0.802955
67     0.115432
68     0.667734
69     0.092112
70     0.203354
71     0.675663
72     0.274895
73     0.092112
74     0.075588
75     0.095163
78     0.439138
79     0.557372
80     0.104866
81     0

## Part (1): Load in our data and conduct basic EDA


We will first load in our dataset below and look at the first few rows. Then, we use the describe function to get a sense of the data. **NOTE: A friendly reminder, this data is different from that of the homework. Please take care to make sure you do not accidentally use this data for the homework!**

In [136]:
df = pd.read_csv('https://github.com/albertw1/data/raw/master/section_6_cancer_data.csv')
df.head()

Y    Gene 1    Gene 2    Gene 3    Gene 4    Gene 5    Gene 6    Gene 7    Gene 8    Gene 9   Gene 10   Gene 11   Gene 12   Gene 13   Gene 14   Gene 15   Gene 16   Gene 17   Gene 18   Gene 19   Gene 20   Gene 21   Gene 22   Gene 23   Gene 24   Gene 25   Gene 26   Gene 27   Gene 28   Gene 29   Gene 30   Gene 31   Gene 32   Gene 33   Gene 34   Gene 35   Gene 36   Gene 37   Gene 38   Gene 39   Gene 40   Gene 41   Gene 42   Gene 43   Gene 44   Gene 45   Gene 46   Gene 47   Gene 48   Gene 49   Gene 50   Gene 51   Gene 52   Gene 53   Gene 54   Gene 55   Gene 56   Gene 57   Gene 58   Gene 59   Gene 60   Gene 61   Gene 62   Gene 63   Gene 64   Gene 65   Gene 66   Gene 67   Gene 68   Gene 69   Gene 70   Gene 71   Gene 72   Gene 73   Gene 74   Gene 75   Gene 76   Gene 77   Gene 78   Gene 79   Gene 80   Gene 81   Gene 82   Gene 83   Gene 84   Gene 85   Gene 86   Gene 87   Gene 88   Gene 89   Gene 90   Gene 91   Gene 92   Gene 93   Gene 94   Gene 95   Gene 96   Gene 97   Gene 98   Gene 99  \
0  0 -0.927178 -0.740039 -0.532016 -1.097891 -0.986673  0.018752 -0.863326  3.959997 -0.398219 -1.097891  0.183420 -1.097891 -1.097891  0.301076 -1.097891  2.172958  0.606712  1.420045  3.485251 -1.097891 -1.097891 -1.097891  1.223614  1.421553  0.732994 -0.871084 -1.097891 -0.087447 -0.087447 -1.097891  1.716598 -1.097891 -0.351254 -1.097891  1.110441 -0.686979  2.552726 -1.097891 -1.097891 -1.022364 -1.097891 -1.097891 -1.097891  0.585757 -0.803376 -1.097891 -1.097891 -0.618410  0.196629 -1.097891 -1.097891 -0.588759 -1.097891  2.747928  0.749564 -0.417653  1.417023 -1.097891  1.835157 -0.840413 -0.803376 -1.097891  0.043598 -0.160175 -0.422572 -1.097891 -0.894733 -1.097891 -0.412758 -0.202598  2.107798 -1.097891 -0.403041  0.630608  0.128733 -0.190845  0.382527 -0.202598  0.604983  0.708381  0.950157  3.432058 -1.097891 -0.796133 -0.259388 -1.097891  1.357764 -1.097891 -1.097891 -1.097891 -1.097891 -0.333080  0.058813  0.082678 -0.977950  0.777542 -0.293378  0.439465 -1.097891   
1  0 -0.835899 -0.835899 -0.585647 -0.835899 -0.329768 -0.835899 -0.835899  4.037788 -0.835899 -0.835899  0.934701 -0.835899 -0.835899  0.437471  2.026290  2.828351  1.291165  2.872351  3.434738 -0.835899 -0.835899 -0.835899 -0.174752 -0.744989 -0.514960 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899  0.111498 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899  2.981215 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.514960 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899  3.243331 -0.661994 -0.835899  0.146187 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899  0.075500 -0.835899 -0.835899 -0.835899 -0.835899 -0.835899  0.243271 -0.835899 -0.275248 -0.835899  1.526308 -0.835899 -0.835899 -0.835899 -0.835899  0.146187 -0.835899  3.180358 -0.835899 -0.835899 -0.835899 -0.835899  2.375943 -0.835899 -0.661994 -0.835899 -0.835899 -0.835899 -0.449153 -0.835899 -0.835899  1.714795 -0.835899  1.159392 -0.835899   
2  0  0.236073  0.252645 -1.154351 -0.372372 -0.338900  1.253467  0.506325  3.868961  0.201932 -1.154351 -1.073012  0.148014 -1.154351  0.576509 -1.154351  0.892058  0.973397 -0.930445  1.619579 -0.245707  0.456109  0.007132  1.331104  0.852357  0.966315  0.049812 -0.372372  0.598677  0.576509 -0.443531  2.016960  0.443080  0.682003 -0.443531  1.402263 -1.154351 -0.275655 -1.154351  0.070377  0.110078  0.007132  0.201932 -1.154351  1.034625 -0.481467 -1.154351 -1.154351 -0.701503  0.884264  0.553749 -0.481467 -0.061178 -1.154351 -1.154351  1.258531  0.345838  2.016960 -0.998755  0.252645 -0.061178  0.494045 -1.154351  0.148014 -0.753241  0.284853 -1.154351 -1.154351  0.166368  0.530367 -0.481467  2.362299 -1.154351  0.876398  1.243246 -1.154351  0.620283  1.410755  0.184335  0.219173  1.431622  1.475809  2.739170 -0.753241  0.236073  0.049812 -0.652723  0.028738 -1.154351  0.711266 -1.154351 -1.154351  0.506325 -1.154351  0.827618  0.028738 -1.154351  0.360426 -1.154351 -1.154351   
3  0 -0.748

In [137]:
df.describe()

Y      Gene 1      Gene 2      Gene 3      Gene 4      Gene 5      Gene 6      Gene 7      Gene 8      Gene 9     Gene 10     Gene 11     Gene 12     Gene 13     Gene 14     Gene 15     Gene 16     Gene 17     Gene 18     Gene 19     Gene 20     Gene 21     Gene 22     Gene 23     Gene 24     Gene 25     Gene 26     Gene 27     Gene 28     Gene 29     Gene 30     Gene 31     Gene 32     Gene 33     Gene 34     Gene 35     Gene 36     Gene 37     Gene 38     Gene 39     Gene 40     Gene 41     Gene 42     Gene 43     Gene 44     Gene 45     Gene 46     Gene 47     Gene 48     Gene 49     Gene 50     Gene 51     Gene 52     Gene 53     Gene 54     Gene 55     Gene 56     Gene 57     Gene 58     Gene 59     Gene 60     Gene 61     Gene 62     Gene 63     Gene 64     Gene 65     Gene 66     Gene 67     Gene 68     Gene 69     Gene 70     Gene 71     Gene 72     Gene 73     Gene 74     Gene 75     Gene 76     Gene 77     Gene 78     Gene 79     Gene 80     Gene 81  \
count  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000   
mean     0.509804   -0.623086   -0.665809   -0.586693   -0.895592   -0.438002   -0.152997   -0.593764    4.361345   -0.553667   -0.951837    0.210988   -0.655313   -0.942147    1.056787    1.048407    2.420894    1.385449    1.984182    3.279895   -0.804082   -0.663843   -0.547087    0.982289    0.420177    0.589138   -0.654416   -0.763602   -0.177263   -0.354601   -0.733137    1.019639   -0.853698   -0.387593   -0.723554    0.333692   -0.870300    2.302401   -0.923628   -0.615913   -0.573766   -0.665838   -0.520763   -0.880360    0.206195   -0.512913   -0.927772   -0.818706   -0.693632   -0.077625   -0.674515   -0.792538   -0.596294   -0.943320    2.282409    0.475842   -0.598812    1.027012   -0.725107    0.342511   -0.816244   -0.278585   -0.498915   -0.177066   -0.528816   -0.262572   -0.948803   -0.852756   -0.833931   -0.218727   -0.653240    1.429048   -0.771873   -0.073120    0.287660    0.001412   -0.551145   -0.686902   -0.494723   -0.368184    0.538564    0.488985   
std      0.502373    0.409905    0.378207    0.497839    0.190989    0.597182    0.789601    0.414790    0.410420    0.399223    0.181319    0.606445    0.469975    0.243418    0.730810    1.528193    0.703898    0.712412    1.423613    0.860988    0.314300    0.455784    0.569803    0.607126    0.735268    0.669660    0.401457    0.292917    0.518869    0.560968    0.427445    0.629158    0.282037    0.527495    0.362698    0.837500    0.241137    1.421855    0.202965    0.667866    0.430846    0.352903    0.511037    0.225416    0.577369    0.537635    0.211668    0.339570    0.372069    0.664546    0.484781    0.311698    0.368384    0.191362    1.739226    0.716093    0.408567    0.581369    0.322546    1.304549    0.285212    0.568969    0.569240    0.597381    0.470440    0.765572    0.203211    0.264404    0.293816    0.563874    0.391422    0.642131    0.350769    0.536137    0.673359    0.957940    0.498118    0.460433    0.428367    0.638959    0.628258    0.855911   
min      0.000000   -1.130098   -1.206937   

As we can see, `Y` here is the response variable (`Y` = 1 is cancer and `Y` = 0 is no cancer) and the 6033 predictor variables are gene expressions. The meaning of each column has no easily interpretable meaning. The way to think about it is that the bigger the value is for a gene, the more likely that gene could lead to someone having cancer. Each row represents a person.

Now, let's split this dataset up into a testing and training set using a 50-50 split. 

In [138]:
np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.5
data_train_ns = df[msk] # ns for non-standarized data, we will standardize it right after this
data_test_ns = df[~msk]
data_train_ns.head()

Y    Gene 1    Gene 2    Gene 3    Gene 4    Gene 5    Gene 6    Gene 7    Gene 8    Gene 9   Gene 10   Gene 11   Gene 12   Gene 13   Gene 14   Gene 15   Gene 16   Gene 17   Gene 18   Gene 19   Gene 20   Gene 21   Gene 22   Gene 23   Gene 24   Gene 25   Gene 26   Gene 27   Gene 28   Gene 29   Gene 30   Gene 31   Gene 32   Gene 33   Gene 34   Gene 35   Gene 36   Gene 37   Gene 38   Gene 39   Gene 40   Gene 41   Gene 42   Gene 43   Gene 44   Gene 45   Gene 46   Gene 47   Gene 48   Gene 49   Gene 50   Gene 51   Gene 52   Gene 53   Gene 54   Gene 55   Gene 56   Gene 57   Gene 58   Gene 59   Gene 60   Gene 61   Gene 62   Gene 63   Gene 64   Gene 65   Gene 66   Gene 67   Gene 68   Gene 69   Gene 70   Gene 71   Gene 72   Gene 73   Gene 74   Gene 75   Gene 76   Gene 77   Gene 78   Gene 79   Gene 80   Gene 81   Gene 82   Gene 83   Gene 84   Gene 85   Gene 86   Gene 87   Gene 88   Gene 89   Gene 90   Gene 91   Gene 92   Gene 93   Gene 94   Gene 95   Gene 96   Gene 97   Gene 98   Gene 99  \
0   0 -0.927178 -0.740039 -0.532016 -1.097891 -0.986673  0.018752 -0.863326  3.959997 -0.398219 -1.097891  0.183420 -1.097891 -1.097891  0.301076 -1.097891  2.172958  0.606712  1.420045  3.485251 -1.097891 -1.097891 -1.097891  1.223614  1.421553  0.732994 -0.871084 -1.097891 -0.087447 -0.087447 -1.097891  1.716598 -1.097891 -0.351254 -1.097891  1.110441 -0.686979  2.552726 -1.097891 -1.097891 -1.022364 -1.097891 -1.097891 -1.097891  0.585757 -0.803376 -1.097891 -1.097891 -0.618410  0.196629 -1.097891 -1.097891 -0.588759 -1.097891  2.747928  0.749564 -0.417653  1.417023 -1.097891  1.835157 -0.840413 -0.803376 -1.097891  0.043598 -0.160175 -0.422572 -1.097891 -0.894733 -1.097891 -0.412758 -0.202598  2.107798 -1.097891 -0.403041  0.630608  0.128733 -0.190845  0.382527 -0.202598  0.604983  0.708381  0.950157  3.432058 -1.097891 -0.796133 -0.259388 -1.097891  1.357764 -1.097891 -1.097891 -1.097891 -1.097891 -0.333080  0.058813  0.082678 -0.977950  0.777542 -0.293378  0.439465 -1.097891   
5   0 -0.319511 -0.106289 -1.085658 -0.997987  0.607362  0.212231 -0.280363  4.052708 -0.044944 -1.085658 -0.448070 -1.085658 -1.085658  0.531298 -1.085658  2.371919  0.430852  1.492998  2.873412 -0.653327 -0.844323  0.166229  1.487406  0.904263  1.348411 -0.044944 -0.712693  0.562482  0.376191 -1.085658  1.878879 -1.085658  0.547022 -0.448070  1.301573 -1.085658  2.323495 -1.085658  0.482445 -1.085658 -0.360400 -0.997987 -1.085658  0.691041  0.945611 -1.085658 -0.106289 -0.138568  0.965609  0.394775 -0.597562 -0.106289 -1.085658  1.059552  1.321939 -0.712693  1.567910 -0.280363  2.122185 -0.360400  0.448373 -1.085658  0.677613 -0.206736 -1.085658 -1.085658 -0.280363 -0.403191  0.779923 -0.242813  0.691041 -1.085658 -0.138568  0.994812 -1.085658  1.041515 -1.085658 -0.403191  0.849818  1.386744  1.374143  3.301069 -0.206736 -0.280363 -0.544985 -0.776156  0.531298 -1.085658 -1.085658 -1.085658  0.394775  0.066689 -1.085658  0.803817 -0.917950 -0.495252  0.297859 -0.917950 -1.085658   
9   0  0.312973 -0.183571 -1.154732 -1.154732 -0.325427  1.338133  0.641860  3.932089  0.253309 -1.154732 -1.002505  0.119433 -1.154732  0.622213 -1.154732  1.200949  0.905982 -0.132953  2.014273 -1.154732  0.382016  0.043477  1.054665  1.363058  1.111875  0.023357 -0.237457 -0.018399  0.689044  0.002740  2.192666 -0.496418  0.538650 -0.389684  1.488249 -1.002505 -1.154732 -1.075154  0.632094 -0.663966 -0.085229  0.082318 -0.618823  0.999594 -0.295062 -0.711689 -1.154732 -0.423768  0.733704 -0.816193 -0.132953  0.082318 -1.154732 -1.154732  1.452255 -0.496418  1.871374 -0.535259  0.119433 -0.018399  0.237703 -1.154732  0.800535 -0.108750  0.137389 -1.154732 -1.154732 -0.873798  0.591848 -1.154732  1.693371 -0.535259  0.591848  1.312440 -1.154732  0.253309 -0.762307  0.237703  1.078021  1.459579  1.804172  2.786420 -1.154732 -0.040087  0.189052 -0.816193  0.395165 -1.154732 -0.108750 -1.075154 -0.762307  0.869832 -1.154732  0.591848  0.119433 -1.154732  0.549564 -1.154732 -1.154732   
12  0 -0

Now let us standarize our data to be between 0 and 1. 

In [139]:
# Your code goes here
data_train = (data_train_ns - data_train_ns.min(axis=0))/(data_train_ns.max(axis=0) - data_train_ns.min(axis=0))
data_test = (data_test_ns - data_train_ns.min(axis=0))/(data_train_ns.max(axis=0) - data_train_ns.min(axis=0))

data_train['Y']=data_train_ns['Y']
data_test['Y']=data_test_ns['Y']
data_train.describe()

Y     Gene 1     Gene 2     Gene 3     Gene 4     Gene 5     Gene 6     Gene 7     Gene 8     Gene 9    Gene 10    Gene 11    Gene 12    Gene 13    Gene 14    Gene 15    Gene 16    Gene 17    Gene 18    Gene 19    Gene 20    Gene 21    Gene 22    Gene 23    Gene 24    Gene 25    Gene 26    Gene 27    Gene 28    Gene 29    Gene 30    Gene 31    Gene 32    Gene 33    Gene 34    Gene 35    Gene 36    Gene 37    Gene 38    Gene 39    Gene 40    Gene 41    Gene 42    Gene 43    Gene 44    Gene 45    Gene 46    Gene 47    Gene 48    Gene 49    Gene 50    Gene 51    Gene 52    Gene 53    Gene 54    Gene 55    Gene 56    Gene 57    Gene 58    Gene 59    Gene 60    Gene 61    Gene 62    Gene 63    Gene 64    Gene 65    Gene 66    Gene 67    Gene 68    Gene 69    Gene 70    Gene 71    Gene 72    Gene 73    Gene 74    Gene 75    Gene 76    Gene 77    Gene 78    Gene 79    Gene 80    Gene 81    Gene 82    Gene 83    Gene 84    Gene 85    Gene 86    Gene 87    Gene 88    Gene 89  \
count  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000  45.000000   
mean    0.555556   0.383577   0.320030   0.378820   0.318071   0.354756   0.396755   0.313066   0.478397   0.412885   0.478030   0.494924   0.335327   0.139823   0.479709   0.561239   0.617575   0.422122   0.622796   0.639505   0.302034   0.279714   0.270741   0.650406   0.498088   0.571854   0.271302   0.295273   0.448398   0.325227   0.248734   0.614359   0.257844   0.328624   0.274973   0.486901   0.254152   0.716429   0.409496   0.230406   0.377514   0.325719   0.349900   0.312920   0.517692   0.324438   0.314912   0.270547   0.317832   0.416715   0.287293   0.351266   0.296703   0.478030   0.667465   0.569652   0.362456   0.535833   0.323748   0.420864   0.336966   0.387970   0.330736   0.416420   0.268304   0.356917   0.478030   0.253282   0.253492   0.356001   0.387557   0.569766   0.272314   0.440194   0.508219   0.425305   0.227521   0.273551   0.340763   0.358209   0.577413   0.491663   0.470968   0.150264   0.322824   0.324471   0.318804   0.610798   0.478030   0.321037   
std     0.502519   0.299458   0.274361   0.295306   0.194051   0.283082   0.334383   0.258170   0.216552   0.280054   0.279078   0.237255   0.283082   0.153812   0.303240   0.379876   0.307605   0.301777   0.302004   0.264247   0.233362   0.255062   0.222415   0.256995   0.265332   0.272165   0.281467   0.228966   0.314848   0.314212   0.224392   0.248219   0.218887   0.325583   0.222870   0.356398   0.198773   0.303473   0.236268   0.256038   0.271662   0.214099   0.296967   0.217155   0.287275   0.271992   0.201110   0.224772   0.282229   0.324382   0.253848   0.259121   0.290691   0.279078   0.351935   0.309640   0.241004   0.277427   0.275182   0.350230   0.242202   0.337550   0.296291   0.348442   0.277196   0.290984   0.279078   0.200526   0.211508   0.308134   0.254170   0.251563   0.205377   0.288597   0.303104   0.342975   0.253842   0.228807   0.223585   0.273457   0.266921   0.327048   0.255894   0.160846   0.311259   0.264103   0.222751   0.302627   0.279078   0.246397   
min     0.000000   

Now to let us subset and create convenient formats for parts in train and test.

In [140]:
X_train = data_train.loc[:, data_train.columns != 'Y'] #Your code goes here
X_test  = data_test.loc[:, data_test.columns != 'Y']
X_train_np = X_train.values # This is to create a numpy array version of the training and testing set.
X_test_np  = X_test.values
y_train = data_train['Y'].values
y_test = data_test['Y'].values

## Part (2): Using one single predictor to fit a Linear Regression Model


Let's use one single predictor variable, `Gene 10` and fit a linear regression model to it. 

In [141]:
import statsmodels.api as sm
X_train_gene_10 = sm.add_constant(X_train["Gene 10"])
X_test_gene_10 = sm.add_constant(X_test["Gene 10"])

ols = OLS(endog=y_train, exog=X_train_gene_10).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     8.306
Date:                Tue, 17 Oct 2017   Prob (F-statistic):            0.00614
Time:                        21:06:20   Log-Likelihood:                -28.407
No. Observations:                  45   AIC:                             60.81
Df Residuals:                      43   BIC:                             64.43
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2092      0.139      1.508      0.139      -0.071       0.489
Gene 10        0.7245      0.251      2.882      0.006       0.218       1.231
==============================================================================
Omnibus:                       17.192   Durbin-Watson:                   0.292
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                3.528
Skew:                          -0.160   Prob(JB):                        0.171
Kurtosis:                       1.666   Cond. No.                         4.51
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Let’s plot our model… 

In [142]:
sns.regplot(x=X_train["Gene 10"], y=y_train)

In [143]:
y_hat_train = ols.predict(X_train_gene_10)
y_hat_test = ols.predict(X_test_gene_10)
print(y_hat_test)

1     0.729194
2     0.282786
3     0.328790
4     0.328811
6     0.249891
7     0.374069
8     0.293342
10    0.559819
11    0.352935
17    0.248222
19    0.830855
20    0.559189
22    0.437803
23    0.406408
24    0.672286
25    0.573603
26    0.560174
29    0.709629
30    0.612260
31    0.637726
32    0.676620
34    0.600042
36    0.423485
37    0.598601
38    0.286982
41    2.277758
42    0.651512
43    0.563009
45    0.777572
49    0.770148
50    0.657317
51    0.721028
53    0.347195
54    0.737462
56    0.416964
59    0.394674
64    0.387458
65    0.427232
67    0.249192
70    0.592169
71    0.588983
73    0.498894
74    0.658137
75    0.640102
76    0.633631
77    0.725309
78    0.706165
79    0.445048
80    0.250237
81    0.737036
83    0.565642
87    0.770051
88    0.766984
91    0.209071
95    0.757744
97    0.762108
98    0.781162
dtype: float64


Do you see anything "wrong" above with the predictions we made for the test set using the linear regression model? Hint: look at entry number 41.

Notice it is possible for us to convert the above values into a binary classification by the following code.

In [144]:
y_hat_train_0_1 = y_hat_train >.5
y_hat_test_0_1 = y_hat_test >.5

print("Training Set Accuracy for Linear Regression: ", accuracy_score(y_train, y_hat_train_0_1))
print("Test Set Accuracy for Linear Regression: ", accuracy_score(y_test, y_hat_test_0_1))

Training Set Accuracy for Linear Regression:  0.666666666667
Test Set Accuracy for Linear Regression:  0.508771929825


## Part (3): Using one single predictor to fit a Logistic Regression Model instead


Now we can instead fit a logistic regression model onto our data and see how things change.

In [145]:
logistic_mod = LogisticRegression(C=1000000, fit_intercept=True) #The C value here is a way to prevent regularization
X_train_mul_wcons = sm.add_constant(X_train["Gene 10"])
X_test_mul_wcons = sm.add_constant(X_test["Gene 10"])
logistic_mod.fit(X_train_mul_wcons, y_train)

y_hat_train_logistic_mod = logistic_mod.predict(X_train_mul_wcons)
y_hat_test_logistic_mod = logistic_mod.predict(X_test_mul_wcons)

print("Training Accuracy for Logistic Regression: ", accuracy_score(y_train, y_hat_train_logistic_mod))
print("Test Accuracy for Logistic Regression: ", accuracy_score(y_test, y_hat_test_logistic_mod))

Training Accuracy for Logistic Regression:  0.666666666667
Test Accuracy for Logistic Regression:  0.526315789474


As we see, the training accuracy is basically the same, but the test accuracy with logsitic regression is better due to being able to better account for the cases where the response variable is very large and very small for `Gene 10`. The takeaway message here is that logistic regresison will outperform a model when your `Y` values tend to be very extreme.

## Part (4): Using all predictor variables in a logistic regression

In [146]:
logistic_mod_all = LogisticRegression(C=100000, fit_intercept=True)
X_train_mul_wcons = sm.add_constant(X_train)
X_test_mul_wcons = sm.add_constant(X_test)

logistic_mod_all.fit(X_train_mul_wcons, y_train)
y_hat_train_logistic_mod_all = logistic_mod_all.predict(X_train_mul_wcons)
y_hat_test_logistic_mod_all = logistic_mod_all.predict(X_test_mul_wcons)
print("Training accuracy for Logistic Regression: ", accuracy_score(y_train, y_hat_train_logistic_mod_all))
print("Test accuracy for Logistic Regression: ", accuracy_score(y_test, y_hat_test_logistic_mod_all))

Training accuracy for Logistic Regression:  1.0
Test accuracy for Logistic Regression:  0.824561403509


In [147]:
predicted = np.round(y_hat_test_logistic_mod_all)
expected = y_test
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

          0       0.86      0.80      0.83        30
          1       0.79      0.85      0.82        27

avg / total       0.83      0.82      0.82        57



In [148]:
pd.crosstab(expected,predicted,margins=True, rownames=['Actual'], colnames=['Predicted'])

Predicted   0   1  All
Actual                
0          24   6   30
1           4  23   27
All        28  29   57

Using this matrix, we can see that of the people who actually **did not** have cancer, 6 were said to have cancer (false positive), while of the people who acutally **did** have cancer, 4 were mistakenly said to not have it (false negative). In biomedical applications, it usually is the case that false negatives tend to have greater consequences. For example, a patient would rather be told they mistakenly had cancer, which might lead to more testing.

## Part (4): Incorporating Bootstrapping in a Logistic Regression Model

The main idea behind bootstrapping is to resample our data many times. For EACH time that we resample our data, we will fit the Logistic Regression model. Then for EACH resampling, we can find an associated confidence interval. Notice how we have many more predictors than observations. Intuitively, that creates a problem in that we do not have enough observations to "represent" each predictor varaible. Hence, the resampling gives us a way to approximate our standard errors. 

The goal here is to find confidence intervals for our logistic regression model using ALL the predictors. You may ask, why don't we just fit a logistic regression, then look at the reported standard errors and confidence intervals? The answer is that with more predictors than observations, the reported confidence intervals will look like NaN. 

### A simple example of why we need a bootstrap when the predictors are greater than our observations:

As an example, we can fit a regularized logistic regression model on ONLY the first 90 genes. The 90 was a random number I chose to be greater than the observations (we could have done 100, 3000, etc). We only did the first 90 as it would take forever to do all 6033 genes. The reason we used the regularization fit was to get the logistic module to report at least some of the NaN's. Notice how many NaN's there are for the confidence intervals reported below, even with something like regularization!

In [152]:
import statsmodels.discrete.discrete_model as dm
model1 = dm.Logit(y_train, X_train_mul_wcons.iloc[:,0:91]).fit_regularized()
print(model1.summary())


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 8.76542346600213e-11
            Iterations: 47
            Function evaluations: 48
            Gradient evaluations: 47
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   45
Model:                          Logit   Df Residuals:                      -46
Method:                           MLE   Df Model:                           90
Date:                Tue, 17 Oct 2017   Pseudo R-squ.:                   1.000
Time:                        21:09:13   Log-Likelihood:            -3.9444e-09
converged:                       True   LL-Null:                       -30.913
                                        LLR p-value:                    0.9898
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------

### The code below helps us conduct a boostrap analysis and be able to obtain confidence interval estimates for each coefficient value!

The above example shows us we need a method to be able to estimate our confidence intervals as it clearly is lacking at only 90 genes. The following code helps us conduct a bootstrap analysis and estimate just how many of our coefficients are significant.

In [153]:
B = 100 # Number of iterations

boot_coefs = np.zeros((X_train.shape[1]+1,B)) # Create empty storage array for later use

for i in range(B):
    # Sampling WITH replacement the indices of a resampled dataset
    sample_index = np.random.choice(range(len(y_train)), size=len(y_train), replace=True)
    X_train_samples = X_train_mul_wcons.values[sample_index]
    y_train_samples = y_train[sample_index]
    
    logistic_mod_boot = LogisticRegression(C=100000, fit_intercept=True)
    logistic_mod_boot.fit(X_train_samples, y_train_samples)
    boot_coefs[:,i] = logistic_mod_boot.coef_

boot_coefs.shape

(6034, 100)

If we wanted to find the 95 percent confidence interval, we can use the `np.percentile` function to find the datapoint that corresponds to the upper and lower bounds. As an example, we have the upper datapoint for the 95% confidence interval. (Why is it 97.5? Think of an interval from 0 to 100, and you want to shade in 95% of that interval, while keeping the length to the left and right of the shading the same. Where does the shading end and begin?)

In [154]:
ci_upper = np.percentile(boot_coefs, 97.5, axis=1)
ci_lower = np.percentile(boot_coefs, 2.5, axis=1)

In [155]:
# ct significant predictors
sig_b_ct = 0

# if ci contains 0, then insignificant
for i in range(len(ci_upper)):
    if ci_upper[i]<0 or ci_lower[i]>0:
        sig_b_ct += 1

print("Significant coefficents at 5pct level = %i / %i" % (sig_b_ct, X_train_np.shape[1]))

Significant coefficents at 5pct level = 1909 / 6033
